# Inference

In [1]:
from src.transformer_lm_prompt import TransformerLanguageModelPrompt
import os
import json
import torch
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
from transformers import BioGptTokenizer
import spacy

nlp = spacy.load("en_ner_bionlp13cg_md")
# nlp = spacy.load("en_core_sci_scibert")
# nlp = spacy.load("en_ner_bc5cdr_md")
# nlp = spacy.load("en_core_sci_lg")
# nlp = spacy.load("en_ner_craft_md")
tokenizer = BioGptTokenizer.from_pretrained("microsoft/biogpt")


/home/tian/mambaforge/envs/BioGPT/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
gold_file = 'data/KD-DTI/raw/test.json'
pmid = 'data/KD-DTI/raw/relis_test.pmid'

with open (gold_file, 'r') as f:
    gold_d = json.load(f)

print(len(gold_d))

# load the pmid
pmids = []
with open (pmid, 'r') as f:
    for line in f:
        line = line.rstrip()
        pmids.append(line)

print(len(pmids))

1159
1159


In [3]:
m = TransformerLanguageModelPrompt.from_pretrained(
    'checkpoints/RE-DTI-BioGPT', 
    'checkpoint_avg.pt', 
    'data/KD-DTI/relis-bin',
    max_len_b=1024,
    max_tokens=12000,)

2023-05-22 04:24:44 | INFO | fairseq.file_utils | loading archive file checkpoints/RE-DTI-BioGPT
2023-05-22 04:24:44 | INFO | fairseq.file_utils | loading archive file data/KD-DTI/relis-bin
2023-05-22 04:24:45 | INFO | src.language_modeling_prompt | dictionary: 42384 types
2023-05-22 04:24:48 | INFO | fairseq.models.fairseq_model | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'user_dir': '../../src', 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False

In [4]:
print(m.cfg)
m.cuda()

{'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'user_dir': '../../src', 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasma'}, 'common_eval': {'_name': None, 'path': None, 'post_process': None, 'quiet': False, 'model_overrides': '{}', 'results_path': None}, 'distributed_training': {'_name': None, 'distributed_world_size': 1, 'distributed_rank': 0, 'distributed_backend': 'n

GeneratorHubInterface(
  (models): ModuleList(
    (0): TransformerLanguageModelPrompt(
      (decoder): TransformerDecoder(
        (dropout_module): FairseqDropout()
        (embed_tokens): Embedding(42393, 1024, padding_idx=1)
        (embed_positions): LearnedPositionalEmbedding(1026, 1024, padding_idx=1)
        (layers): ModuleList(
          (0-23): 24 x TransformerDecoderLayerBase(
            (dropout_module): FairseqDropout()
            (self_attn): MultiheadAttention(
              (dropout_module): FairseqDropout()
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (activation_dropout_module): FairseqDropout()
            (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwis

In [5]:
from inference_tokens_in_motipleNER import predict
batch_output = []

In [6]:
for n in range(0, len(pmids)):
    batch_output.append(predict(m, n, pmids, gold_d, min_prob=0.02, tokenizer=tokenizer, nlp=nlp))
    if n % 20 == 0:
        torch.save(batch_output, f"batch_output_{n}.pt")

start processing: 1/1159...


/home/tian/Projects/BioGPT/inference_tokens_in_motipleNER.py:134: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  current_output = [torch.tensor(token_id).cpu() for token_id in tokens]


final outputs:  the interaction between monoamine oxidase and monoamine oxidase is inhibitor .
1 / 1159
Elapsed time: 1.5587589740753174s
start processing: 2/1159...
final outputs:  the interaction between cyclosporine a and signal transducers is inhibitor .
2 / 1159
Elapsed time: 1.2926304340362549s
start processing: 3/1159...
final outputs:  the interaction between bradykinin <unk> and bradykinin <unk> is inhibitor .
3 / 1159
Elapsed time: 1.2942290306091309s
start processing: 4/1159...
final outputs:  the interaction between adenosine <unk> monophosphate and adenosine <unk> monophosphate is agonist .
4 / 1159
Elapsed time: 1.2520663738250732s
start processing: 5/1159...
final outputs:  the interaction between cell and cell is inhibitor .
5 / 1159
Elapsed time: 1.2307078838348389s
start processing: 6/1159...
final outputs:  the interaction between glutamate receptor 4 and glutamate receptor 4 is modulator .
6 / 1159
Elapsed time: 1.1695218086242676s
start processing: 7/1159...
final 

In [7]:
torch.save(batch_output, f"batch_output_multipleNER_bionlp13cg.pt")

In [8]:
len(batch_output)

1159

In [9]:
outputs = [m.decode(hypos) for hypos in batch_output]
with open(f"{'examples/RE-DTI/generate_checkpoint_optimized_3-multipleNER_bionlp13cg.pt'}", "w", encoding='utf8') as fw:
    for i in range(len(outputs)):
        fw.write(outputs[i] + '\n')

sed -i "s/@@ //g" generate_checkpoint_optimized_3-multipleNER_bionlp13cg.pt

perl ../../mosesdecoder/scripts/tokenizer/detokenizer.perl -l en -a < generate_checkpoint_optimized_3-multipleNER_bionlp13cg.pt > generate_checkpoint_optimized_3-multipleNER_bionlp13cg.pt.detok

# post process

In [10]:
import os
import sys
import re
import json

In [11]:
out_file = 'examples/RE-DTI/generate_checkpoint_optimized_3-multipleNER_bionlp13cg.pt.detok'

In [12]:
prefix = [
    '(learned[0-9]+ )+',
    'we can conclude that',
    'we have that',
    'in conclusion,',
    ]

def strip_prefix(line: str) -> str:
    """Removes the prefix from the line."""
    # search for the prefix in the line
    for p in prefix:
        res = re.search(p, line)
        
        if res is not None:
            # remove the prefix from the line
            line = re.split(p, line)[-1].strip()
            break
    return line

def split_sentence(line):
    """Splits the line into sentences by semicolon."""
    sentences = re.split(r"; ", line)
    return sentences

def convert_relis_sentence(sentence):
    # match sentences of the form "the interaction between X and Y is Z"
    # and return the tuple (X, Z, Y)
    ans = None
    segs = re.match(r"the interaction between (.*) and (.*) is (.*)", sentence)
    if segs is not None:
        segs = segs.groups()
        ans = (segs[0].strip(), segs[2].strip(), segs[1].strip())
    return ans

def converter(sample, head_idx=0, rel_idx=1, tail_idx=2):
    ret = {
        "triple_list_gold": [], 
        "triple_list_pred": [], 
        "new": [], 
        "lack": [], 
        "id": [0]
    }
    for item in sample:
        ret["triple_list_pred"].append(
            {"subject": item[head_idx], 
            "relation": item[rel_idx], 
            "object": item[tail_idx]}
        )
    return ret


all_lines = []
with open(out_file, "r", encoding="utf8") as fr:
    for line in fr:
        e = line.strip()
        if len(e) > 0 and e[-1] == ".": # if the last character of the line is a period, remove it
            all_lines.append(e[:-1])
        else:
            all_lines.append(e)

print(f'Number of lines: {len(all_lines)}')
hypothesis = []
cnt = 0
fail_cnt = 0

# i is the index of the line in the input file
for i, line in enumerate(all_lines):
    # cnt is the number of lines in the input file
    cnt += 1
    ret = []
    # convert the "& amp" if it is in line to "&"
    line = line.replace(" & amp; ", "&")
    strip_line = strip_prefix(line)
    sentences = split_sentence(strip_line)
    for sen in sentences:
        ans = convert_relis_sentence(sen)
        if ans is not None:
            ret.append(ans)
    if len(ret) > 0:
        hypothesis.append(ret)
    else:
        hypothesis.append([("failed", "failed", "failed")])
        fail_cnt += 1
        print("Failed:id:{}, line:{}".format(i+1, line))


ret_formatted = []
for i in range(len(hypothesis)):
    ret_formatted.append(converter(hypothesis[i]))

with open(f"{out_file}.extracted.json", "w", encoding="utf8") as fw:
    for eg in ret_formatted:
        print(json.dumps(eg), file=fw)


print(f"failed = {fail_cnt}, total = {cnt}")

Number of lines: 1159
Failed:id:201, line:the interaction between carbonic anhydrase (carbonic anhydrase (carbonic anhydrase (carbonic anhydrase (carbonic anhydrase (and carbonic anhydrase (carbonic anhydrase (carbonic anhydrase (carbonic anhydrase (carbonic anhydrase (is positive allosteric modulator; ca)) ix and carbonic anhydrase (carbonic anhydrase (carbonic anhydrase (carbonic anhydrase (carbonic anhydrase (is positive allosteric modulator;
Failed:id:225, line:the interaction between cyclooxygenase and lipoxygenase (leukocyte lipoxygenase (leukocyte lipoxygenase (is unknown; lox)) rat paw leukocyte lipoxygenase (leukocyte lipoxygenase (and rat paw leukocyte lipoxygenase (leukocyte lipoxygenase (is unknown; lox)) rat paw leukocyte lipoxygenase (and rat paw leukocyte lipoxygenase (is unknown;
Failed:id:289, line:the interaction between endothelin converting enzyme (endothelin converting enzyme (endothelin converting enzyme (endothelin converting enzyme (endothelin converting enzyme 

# Eval

In [14]:
import re
import json
import sys
import os

In [16]:
pred_file = 'examples/RE-DTI/generate_checkpoint_optimized_3-multipleNER_bionlp13cg.pt.detok.extracted.json'
gold_file = 'data/KD-DTI/raw/test.json'
pmids_file = 'data/KD-DTI/raw/relis_test.pmid'

In [17]:
def normalize_name(s: str):
    s = s.strip()

    # normalize roman type id at end of string
    num2roman = {"0": "0", "1": "I", "2": "II", "3": "III", "4": "IV", "5": "V", "6": "VI", "7": "VII", "8": "VIII", "9": "IX"}
    if len(s) > 2 and s[-1].isnumeric() and not s[-2].isnumeric() and s[-1] in num2roman:
        tmps = list(s)
        s = ''.join(tmps[:-1]) + num2roman[tmps[-1]]

    # remove useless end string
    s = s.replace("-type", '')
    
    # remove non-alphanumeric characters
    return re.sub('[^a-zA-Z0-9]+', '', s)

# e.p. for normalize_name

print(f"trace amine-associated receptor 1 (taar1)\n{normalize_name('trace amine-associated receptor 1 (taar1)')}\n\ntrace amine-associated receptor 1\n{normalize_name('trace amine-associated receptor 1')}\n\ntrace amine-associated receptor 21\n{normalize_name('trace amine-associated receptor 21')}")

def rm_abbr(tgt_set):
    """ remove abbreviation in the brackets of entity, eg: aaa (bb) -> aaa """
    def rm(s):
        s = s.strip()
        if "(" in s and s[-1] == ')':  # entity end with a bracketed short cut
            return normalize_name(s[:s.rfind("(")].strip())
        else:
            return normalize_name(s)

    tgt_set = list(tgt_set)
    if tgt_set and type(tgt_set[0]) in [tuple, list]:  # process triples
        return set([(rm(tp[0]), rm(tp[1]), rm(tp[2])) for tp in tgt_set])
    else:  # process entities
        return set([rm(e) for e in tgt_set])
    
def get_abbr(tgt_set):
    """ extract abbreviation in the brackets of entity, eg: aaa (bb) -> bb """
    def rm(s):
        s = s.strip()
        if "(" in s and s[-1] == ')':
            return normalize_name(s[s.rfind("(")+1:-1].strip())
        else:
            return normalize_name(s)

    tgt_set = list(tgt_set)
    if tgt_set and type(tgt_set[0]) in [tuple, list]:  # process triples
        return set([(rm(tp[0]), rm(tp[1]), rm(tp[2])) for tp in tgt_set])
    else:  # process entities
        return set([rm(e) for e in tgt_set])
    

def acc(pred_set, gold_set):
    """ Multi-label style acc """
    tp_num = len(pred_set & gold_set)
    return int(pred_set == gold_set) if len(gold_set) == 0 else 1.0 * tp_num / len(pred_set | gold_set)


def precision(pred_set, gold_set):
    """ Multi-label style precision """
    tp_num = len(pred_set & gold_set)
    return int(pred_set == gold_set) if len(pred_set) == 0 else 1.0 * tp_num / len(pred_set)


def recall(pred_set, gold_set):
    """ Multi-label style recall """
    tp_num = len(pred_set & gold_set)
    return int(pred_set == gold_set) if len(gold_set) == 0 else 1.0 * tp_num / len(gold_set)


def normed_eval(pred_set, gold_set, metric):
    """ Both body and abbreviation match are considered correct """
    abbr_pred_set, abbr_gold_set = get_abbr(pred_set), get_abbr(gold_set)
    rm_pred_set, rm_gold_set = rm_abbr(pred_set), rm_abbr(gold_set)
    return max(metric(abbr_pred_set, abbr_gold_set), metric(rm_pred_set, rm_gold_set))


def get_f1(p, r):
    return 0 if (p + r) == 0 else (2.0 * p * r / (p + r))


def ave(scores):
    return 1.0 * sum(scores) / len(scores)




def do_eval(preds, pmids, golden):
    ret = []
    num_pred, num_gold, num_missing = 0, 0, 0
    all_f1, p, r, d_acc, t_acc, i_acc = [], [], [], [], [], []
    all_pred_triple, all_pred_d, all_pred_t, all_pred_i, all_gold_triple, all_gold_d, all_gold_t, all_gold_i = [], [], [], [], [], [], [], [],

    for pred, idx in zip(preds, pmids):
        # initialize a set for each entity type (drug, target, interaction) for the prediction and gold standard
        gold_d_set, gold_t_set, gold_i_set, gold_set = set(), set(), set(), set()
        pred_d_set, pred_t_set, pred_i_set, pred_set = set(), set(), set(), set()
        
        # if the prediction is not empty and if the model did not fail to extract any triple
        if pred["triple_list_pred"] and pred["triple_list_pred"][0]["subject"] != 'failed':
            # loop over all triples and add the entities to the respective sets
            for tp in pred["triple_list_pred"]:
                d = tp["subject"].strip().lower().replace(' ', '')
                t = tp["object"].strip().lower().replace(' ', '')
                i = tp["relation"].strip().lower().replace(' ', '')

                pred_d_set.add(d)
                pred_t_set.add(t)
                pred_i_set.add(i)
                pred_set.add((d, t, i))
        # if the paper is not in the golden set
        if idx not in golden:
            # increase the number of missing papers
            num_missing += 1
            # print a message
            print("----Missing:", idx)
            # skip this paper
            continue
        # if there are triples in the golden set
        if golden[idx]["triples"]:
            # loop over all triples and add the entities to the respective sets
            for tp in golden[idx]["triples"]:
                d = tp["drug"].strip().lower().replace(' ', '')
                t = tp["target"].strip().lower().replace(' ', '')
                i = tp["interaction"].strip().lower().replace(' ', '')
                gold_d_set.add(d)
                gold_t_set.add(t)
                gold_i_set.add(i)
                gold_set.add((d, t, i))

        # sample level eval
        p.append(normed_eval(pred_set, gold_set, metric=precision))
        r.append(normed_eval(pred_set, gold_set, metric=recall))
        all_f1.append(get_f1(p[-1], r[-1]))
        d_acc.append(normed_eval(pred_d_set, gold_d_set, metric=acc))
        t_acc.append(normed_eval(pred_t_set, gold_t_set, metric=acc))
        i_acc.append(normed_eval(pred_i_set, gold_i_set, metric=acc))

        # onto level eval
        all_pred_d.extend(pred_d_set)
        all_pred_t.extend(pred_t_set)
        all_pred_i.extend(pred_i_set)
        all_pred_triple.extend(pred_set)
        all_gold_d.extend(gold_d_set)
        all_gold_t.extend(gold_t_set)
        all_gold_i.extend(gold_i_set)
        all_gold_triple.extend(gold_set)
        
        # if len(gold_set) < len(golden[idx]["triples"]):
            # print("Duplicate extists, ori", golden[idx]["triples"], gold_set)

        num_pred += len(pred_set)
        num_gold += len(gold_set)

        ret.append({
            "pmid": idx,
            "title": golden[idx]["title"] if "title" in golden[idx] else None,
            "abstract": golden[idx]["abstract"],
            "d_pred_gold": [d_acc[-1], list(pred_d_set), list(gold_d_set)],
            "t_pred_gold": [t_acc[-1], list(pred_t_set), list(gold_t_set)],
            "i_pred_gold": [i_acc[-1], list(pred_i_set), list(gold_i_set)],
            "all_pred_gold": [all_f1[-1], list(pred_set), list(gold_set)],
        })


    print("num sample", len(all_f1), "missing", len(preds) - len(all_f1), "num_gold tp", num_gold, "num_pred", num_pred)

    # Note: we adopt multi-label metrics following: http://129.211.169.156/publication/tkde13rev.pdf
    print("Sample: acc d: {:.4f}\tt:{:.4f}\ti: {:.4f}\ntp p: {:.4f}\ttp r: {:.4f}\ttp micro f1: {:.4f}\ttp macro f1: {:.4f} ".format(
        ave(d_acc), ave(t_acc), ave(i_acc), ave(p), ave(r), ave(all_f1), get_f1(ave(p), ave(r))))

    # Ontology evaluation_scripts
    all_p, all_r = normed_eval(set(all_pred_triple), set(all_gold_triple), metric=precision), normed_eval(set(all_pred_triple), set(all_gold_triple), metric=recall)
    d_p, d_r = normed_eval(set(all_pred_d), set(all_gold_d), metric=precision), normed_eval(set(all_pred_d), set(all_gold_d), metric=recall)
    t_p, t_r = normed_eval(set(all_pred_t), set(all_gold_t), metric=precision), normed_eval(set(all_pred_t), set(all_gold_t), metric=recall)
    i_p, i_r = normed_eval(set(all_pred_i), set(all_gold_i), metric=precision), normed_eval(set(all_pred_i), set(all_gold_i), metric=recall)

    print("Ontology: f1 d: {:.4f}\tt:{:.4f}\ti: {:.4f}\t \nall p: {:.4f}\tall r: {:.4f}\tonto f1: {:.4f}".format(
        get_f1(d_p, d_r), get_f1(t_p, t_r), get_f1(i_p, i_r), all_p, all_r, get_f1(all_p, all_r)
    ))
    return ret


preds = []
with open(pred_file) as reader:
    for line in reader:
        preds.append(json.loads(line))

with open(gold_file) as reader:
    golden = json.load(reader)

with open(pmids_file) as reader:
    if '.json' in pmids_file:
        pmids = json.load(reader)
    else:
        pmids = []
        for line in reader:
            pmids.append(line.strip())

trace amine-associated receptor 1 (taar1)
traceamineassociatedreceptor1taar1

trace amine-associated receptor 1
traceamineassociatedreceptorI

trace amine-associated receptor 21
traceamineassociatedreceptor21


In [18]:
print("\n====File: ", os.path.basename(pred_file))


====File:  generate_checkpoint_optimized_3-multipleNER_bionlp13cg.pt.detok.extracted.json


In [19]:
preds[0]

{'triple_list_gold': [],
 'triple_list_pred': [{'subject': 'monoamine oxidase',
   'relation': 'inhibitor',
   'object': 'monoamine oxidase'}],
 'new': [],
 'lack': [],
 'id': [0]}

In [20]:
# new
result = do_eval(preds, pmids, golden)

last_pos = pred_file.rfind('.json')
res_file_name = pred_file[:last_pos] + '.eval_res.json'

num sample 1159 missing 0 num_gold tp 1567 num_pred 1174
Sample: acc d: 0.1241	t:0.1726	i: 0.6304
tp p: 0.0190	tp r: 0.0175	tp micro f1: 0.0180	tp macro f1: 0.0182 
Ontology: f1 d: 0.1269	t:0.1413	i: 0.5667	 
all p: 0.0256	all r: 0.0195	onto f1: 0.0221


In [31]:
result = do_eval(preds, pmids, golden)

last_pos = pred_file.rfind('.json')
res_file_name = pred_file[:last_pos] + '.eval_res.json'

num sample 1159 missing 0 num_gold tp 1567 num_pred 2250
Sample: acc d: 0.6175	t:0.6067	i: 0.8667
tp p: 0.4009	tp r: 0.3980	tp micro f1: 0.3851	tp macro f1: 0.3994 
Ontology: f1 d: 0.6093	t:0.5122	i: 0.7556	 
all p: 0.2518	all r: 0.3735	onto f1: 0.3008


In [42]:
with open(res_file_name, 'w') as writer:
    json.dump(result, writer, indent=2)

See the comparation of the outputs in:

examples/RE-DTI/generate_checkpoint_avg.pt.detok.extracted.eval_res.json

# Analysis

In [2]:
from transformers import BioGptTokenizer
tokenizer = BioGptTokenizer.from_pretrained("microsoft/biogpt")

/home/tian/mambaforge/envs/BioGPT/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
pred_file = 'examples/RE-DTI/generate_checkpoint_avg.pt.detok.extracted.json'
# gold_file = 'data/KD-DTI/raw/test.json'
gold_file = 'data/KD-DTI/raw/train.json'
# pmids_file = 'data/KD-DTI/raw/relis_test.pmid'
pmids_file = 'data/KD-DTI/raw/relis_train.pmid'

In [5]:
import json

preds = []
with open(pred_file) as reader:
    for line in reader:
        preds.append(json.loads(line))

with open(gold_file) as reader:
    golden = json.load(reader)

with open(pmids_file) as reader:
    if '.json' in pmids_file:
        pmids = json.load(reader)
    else:
        pmids = []
        for line in reader:
            pmids.append(line.strip())

In [9]:
golden[pmids[0]]

{'title': 'Beta-subunits co-determine the sensitivity of rat neuronal nicotinic receptors to antagonists.',
 'abstract': "We have investigated the effect of 4 ganglionic cholinergic antagonists (hexamethonium, mecamylamine, pentolinium, trimetaphan) on rat alpha 3 beta 2 and alpha 3 beta 4 neuronal nicotinic acetylcholine receptors (nAChRs) expressed in Xenopus oocytes. Current responses were elicited by fast application of acetylcholine on voltage-clamped oocytes (holding potentialVh = -80mV). Concentration-inhibition curves were used to get estimates of IC50, the antagonist concentration yielding 50% reduction of the peak current. The KB's of the antagonists were calculated using estimates of the apparent KD of acetylcholine. The order of affinity of the antagonists was similar for both receptor subtypes: mecamylamine approximately pentolinium &gt; hexamethonium &gt; trimetaphan. However, alpha 3 beta 4 neuronal nAChRs were 9 to 22 times more sensitive to each of the 4 antagonists th

In [94]:
preds[0]

{'triple_list_gold': [],
 'triple_list_pred': [{'subject': 'isopsoralen',
   'relation': 'inhibitor',
   'object': 'monoamine oxidase type b (mao-b)'},
  {'subject': 'isopsoralen',
   'relation': 'inhibitor',
   'object': 'monoamine oxidase type a (mao-a)'}],
 'new': [],
 'lack': [],
 'id': [0]}

In [100]:
num_pred_D = 0
num_pred_T = 0
num_gold_D = 0
num_gold_T = 0


num_predD_in_original = 0
num_predT_in_original = 0
# # if there are more than 50% prefix tokens of a span is in the original text, count it
# num_predD_over50_in_original = 0
# num_predT_over50_in_original = 0
# partial in the golden means the model didn't generate the whole span.
num_goldD_in_original = 0

predT_not_in_original_pmid_predTs = []

for id, pred in zip(pmids, preds):
    # only use lower() to nomalize the text
    pred_Ds = list(set([triple['subject'].lower() for triple in pred['triple_list_pred']]))
    pred_Ts = list(set([triple['object'].lower() for triple in pred['triple_list_pred']]))
    gold_Ds = list(set([triple['drug'].lower() for triple in golden[id]['triples']]))
    gold_Ts = list(set([triple['target'].lower() for triple in golden[id]['triples']]))
    text = (golden[id]['title'] + " " + golden[id]['abstract']).lower()
    
    num_pred_D += len(pred_Ds)
    num_pred_T += len(pred_Ts)
    num_gold_D += len(gold_Ds)
    num_gold_T += len(gold_Ts)

    predT_not_in_original = []

    # count for if pred_D in the original text
    for pred_D in pred_Ds:
        if pred_D is not None:
            if pred_D in text:
                num_predD_in_original += 1

    # count for if pred_T in the original text
    for pred_T in pred_Ts:
        if pred_T is not None:
            if pred_T in text:
                num_predT_in_original += 1
            else:
                predT_not_in_original.append(pred_T)


    # count for if god_D in the original text
    for gold_D in gold_Ds:
        if gold_D in text:
            num_goldD_in_original += 1

    # count for if god_T in the original text
    for gold_T in gold_Ts:
        if gold_T in text:
            num_goldT_in_original += 1


    if predT_not_in_original:
        predT_not_in_original_pmid_predTs.append({
            "id": id,
            "predT_not_in_original": predT_not_in_original
            })
    # pred_D_ids = [tokenizer.encode(pred_d, add_special_tokens=False) for pred_d in pred_Ds]
    # pred_T_ids = [tokenizer.encode(pred_t, add_special_tokens=False) for pred_t in pred_Ts]
    # gold_D_ids = [tokenizer.encode(gold_d, add_special_tokens=False) for gold_d in gold_Ds]
    # gold_T_ids = [tokenizer.encode(gold_t, add_special_tokens=False) for gold_t in gold_Ts]
    # text_id = tokenizer.encode(text, add_special_tokens=False)

    # # count for if pred_D in the original text
    # for pred_d_id in pred_D_ids:
    #     length = len(pred_d_id)
    #     for i in range(len(text_id)):
    #         for j in range(length):
    #             if text_id[i+j+1] == pred_d_id[:j+1]:
    #                 if j+1 == length:
    #                     num_predD_in_original += 1
    #             else:
    #                 if j+1 > length/2:
    #                     num_predD_over50_in_original += 1
    #                 break
    #         if i + length +1 >= len(text_id):
    #             break
    
    # # count for if pred_T in the original text
    # for pred_t_id in pred_T_ids:
    #     length = len(pred_t_id)
    #     for i in range(len(text_id)):
    #         for j in range(length):
    #             if text_id[i+j+1] == pred_t_id[:j+1]:
    #                 if j+1 == length:
    #                     num_predT_in_original += 1
    #             else:
    #                 if j+1 > length/2:
    #                     print(tokenizer.decode(pred_t_id[:j+1]))
    #                     num_predT_over50_in_original += 1
    #                 break
    #         if i + length +1 >= len(text_id):
    #             break
    
    # # count for if gold_T in the original text
    # for gold_t_id in gold_T_ids:
    #     length = len(gold_t_id)
    #     for i in range(len(text_id)):
    #         for j in range(length):
    #             if text_id[i+j+1] == gold_t_id[:j+1]:
    #                 if j+1 == length:
    #                     num_predT_in_golden += 1
    #             else:
    #                 break
    #         if i + length +1 >= len(text_id):
    #             break
    
    # # count for if gold_D in the original text
    # for gold_d_id in gold_D_ids:
    #     length = len(gold_d_id)
    #     for i in range(len(text_id)):
    #         for j in range(length):
    #             if text_id[i+j+1] == gold_d_id[:j+1]:
    #                 if j+1 == length:
    #                     num_predD_in_golden += 1
    #             else:
    #                 break
    #         if i + length +1 >= len(text_id):
    #             break

train_dataset

In [12]:
len(golden) == len(pmids)

True

In [15]:
num_gold_D = 0
num_gold_T = 0


num_predD_in_original = 0
num_predT_in_original = 0
# # if there are more than 50% prefix tokens of a span is in the original text, count it
# num_predD_over50_in_original = 0
# num_predT_over50_in_original = 0
# partial in the golden means the model didn't generate the whole span.
num_goldD_in_original = 0
num_goldT_in_original = 0

for id in pmids:
    # only use lower() to nomalize the text
    gold_Ds = list(set([triple['drug'].lower() for triple in golden[id]['triples']]))
    gold_Ts = list(set([triple['target'].lower() for triple in golden[id]['triples']]))
    text = (golden[id]['title'] + " " + golden[id]['abstract']).lower()
    
    num_gold_D += len(gold_Ds)
    num_gold_T += len(gold_Ts)

    predT_not_in_original = []

    # count for if god_D in the original text
    for gold_D in gold_Ds:
        if gold_D in text:
            num_goldD_in_original += 1

    # count for if god_T in the original text
    for gold_T in gold_Ts:
        if gold_T in text:
            num_goldT_in_original += 1

In [16]:
# print num_predD_in_original, num_predT_in_original, num_predD_over50_in_original, num_predT_over50_in_original, num_predD_in_golden = 0, num_predT_in_golden with their lables
# print("num_predD_in_original: ", num_predD_in_original)
# print("num_predT_in_original: ", num_predT_in_original)
print("num_goldD_in_original: ", num_goldD_in_original)
print("num_goldT_in_original: ", num_goldT_in_original)
# print("num__pred_D: ", num_pred_D)
# print("num_pred_T: ", num_pred_T)
print("num_gold_D: ", num_gold_D)
print("num_gold_T: ", num_gold_T)

num_goldD_in_original:  8369
num_goldT_in_original:  1535
num_gold_D:  19426
num_gold_T:  17179


In [17]:
# print(f"For pred_D:\n{num_predD_in_original} / {num_pred_D} : {round(num_predD_in_original/num_pred_D, 4)}")
# print(f"For pred_T:\n{num_predT_in_original} / {num_pred_T} : {round(num_predT_in_original/num_pred_T, 4)}")
print(f"For gold_D:\n{num_goldD_in_original} / {num_gold_D} : {round(num_goldD_in_original/num_gold_D, 4)}")
print(f"For gold_T:\n{num_goldT_in_original} / {num_gold_T} : {round(num_goldT_in_original/num_gold_T, 4)}")

For gold_D:
8369 / 19426 : 0.4308
For gold_T:
1535 / 17179 : 0.0894


In [101]:
with open('analysis/predT_not_in_original_pmid_predTs.json', 'w') as writer:
    json.dump(predT_not_in_original_pmid_predTs, writer, indent=2)